In [6]:
import manifold1
import numpy as np
import matplotlib.pyplot as plt
plt.rc('text', usetex=False)
import math
from ripser import ripser
from persim import plot_diagrams


In [ ]:
import matplotlib.pyplot as plt
plt.rc('text', usetex=False)

def plot_points(points, ax, color='black', s=1, title='title', n = 1.5):
    ax.scatter(points[:, 0], points[:, 1], points[:, 2], c=color, s=s)
    ax.set_xlabel('X', fontsize=8, rotation=20, labelpad=2)
    ax.set_ylabel('Y', fontsize=8, rotation=20, labelpad=2)
    ax.set_zlabel('Z', fontsize=8, rotation=20, labelpad=2)
    ax.set_xticks([ -2, -1, 0, 1, 2])
    ax.set_yticks([ -2, -1, 0, 1, 2])
    ax.set_zticks([ -2, -1, 0, 1, 2])
    ax.set_title(title, fontsize=14)
    ax.set_xlim(-n, n)
    ax.set_ylim(-n, n)
    ax.set_zlim(-n, n)

def plot_figure(X, title, n):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    plot_points(X, ax, title=title, n=n)
    plt.show()

***Geometric Anomaly Detection Function***

In [4]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
from ripser import ripser

def geometric_anomaly_detection(P, r, s, k):
    print('Starting geometric anomaly detection')
    # Initialize the subsets of points Pman, Pbnd, and Pint
    Pman, Pbnd, Pint = set(), set(), set()
    
    # Create a NearestNeighbors instance and fit it to the dataset P
    nn = NearestNeighbors(radius=s)
    nn.fit(P)
    print('Nearest Neighbors fitted')
    
    # Iterate over all points in P
    for i, y in enumerate(P):
        print('Processing point', i)
        # Find the annulus Ay for the point y
        indices = nn.radius_neighbors([y], radius=s, return_distance=False)[0]
        Ay = P[indices]
        Ay = Ay[np.linalg.norm(Ay - y, axis=1) >= r]
    
        # Compute persistent cohomology for the annulus Ay
        diagrams = ripser(Ay)['dgms']
        
    
        
        
        # Calculate Ny, the number of sufficiently long intervals in the barcode for dimension (k-1)
        Ny = sum(1 for interval in diagrams[k-1] if interval[1] - interval[0] > (s - r))
    
        
        # Classify the point y based on Ny
        if Ny == 0:
            Pbnd.add(i)
        elif Ny == 1:
            Pman.add(i)
        else:
            Pint.add(i)

    print('Geometric anomaly detection complete')
    
    # Return the partition of P into subsets Pman, Pbnd, and Pint
    return Pman, Pbnd, Pint



***Function Run GAD function + plot***

In [8]:
def plot_P(X, s, k = 2, sizeP="K"):
    # Set the parameters for the geometric anomaly detection algorithm
    r = 5*s/8
    # Run the geometric anomaly detection algorithm
    Pman, Pbnd, Pint = geometric_anomaly_detection(X, r, s, k)
    Pman = X[list(Pman)]
    Pbnd = X[list(Pbnd)]
    Pint = X[list(Pint)]
    print('Manifold points:', len(Pman))
    print('Boundary points:', len(Pbnd))
    print('Interior points:', len(Pint))
    #save subsets of points in files .npy
    np.save('Pman'+str(sizeP), Pman)
    np.save('Pbnd'+str(sizeP), Pbnd)
    np.save('Pint'+str(sizeP), Pint)

    #set string for title of plot : sizeP + Manifold points, Boundary points, Intersection points
    title_Pman = str(sizeP) + ' Manifold points'
    title_Pbnd = str(sizeP) + ' Boundary points'
    title_Pint = str(sizeP) + ' Intersection points'

    # Plot the results of the geometric anomaly detection algorithm
    plot_figure(Pman, title_Pman, 2)
    plot_figure(Pbnd, title_Pbnd, 2)
    plot_figure(Pint, title_Pint, 2)

***Generate Spheres with adjusted sizes***

In [ ]:
import matplotlib.pyplot as plt
plt.rc('text', usetex=False)


def sphere_points(nPoints, sizeAdd = 0, filename = "sphere_points.npy"):
    X1 = manifold1.Sphere.sample(nPoints, 2) # sample 1000 points from a 2-sphere
    X1 -= [.5, .5, 0]
    X2 = manifold1.Sphere.sample(nPoints, 2) # sample 1000 points from a 3-sphere
    X2 += [.5, .5, 0]
    # concatenate the two point clouds
    X = np.concatenate((X1, X2))
    print(X.shape)
    
    data = X * sizeAdd
    np.save(filename, data)
    return data
    

X = sphere_points(1000, 1.7, 'sp1000.npy')
Y = sphere_points(3000, 1.7, 'sp3000.npy')
Z = sphere_points(5000, 1.7, 'sp5000.npy')
